In [1]:
import nosbench
from nosbench.program import Program, Instruction, Pointer
from nosbench.function import Function, interpolate, bias_correct, clip, size
from nosbench.utils import prune_program
import torch

import pprint
import math

from neps.space.new_space import space

from functools import partial

import numpy as np
import pytest

import neps
import neps.space.new_space.space as space
import neps.space.new_space.bracket_optimizer as new_bracket_optimizer

MAX_EPOCHS_PER_CONFIG = 20          # As per the NOSBench paper
AVAILABLE_VARIABLE_SLOTS = 11       # As per the NOSBench paper


def _function_writer(operation: callable, *args, store: int) -> tuple[Function, list[Pointer], Pointer]:
    return Function(operation, len(args)), [Pointer(var) if isinstance(var, int) else Pointer(var()) for var in args], store
    
def program_compiler(instruction: tuple[callable, list[Pointer], Pointer], *args) -> Program:
    if len(args) == 0:
        return Program([Instruction(instruction[0], inputs=instruction[1], output=instruction[2])])
    else:
        return Program(program_compiler(instruction) + sum(args,[]))

class nosBench(space.Pipeline):
    _UNARY_FUN = space.Categorical(
        choices=(torch.square, torch.exp, torch.log, torch.sign, torch.sqrt, torch.abs, torch.norm, torch.sin, torch.cos, torch.tan, torch.asin, torch.acos, torch.atan, torch.mean, torch.std, size,)
    )

    _BINARY_FUN = space.Categorical(
        choices=(clip, torch.div, torch.mul, torch.add, torch.sub, torch.minimum, torch.maximum, torch.heaviside)
    )

    _TERNARY_FUN = space.Categorical(
        choices=(interpolate, bias_correct,)
    )

    _CONST = space.Integer(3, 8) # 0-2 are reserved for weights, gradient, and optimizer step
    _VAR = space.Integer(9, AVAILABLE_VARIABLE_SLOTS+9) # And 3-8 are reserved for the constants

    # Is this the right way to handle constants?
    _PARAMS = space.Categorical(
        choices=(
            0, # weights
            1, # gradient
            2, # optimizer step
        )
    )

    _POINTER = space.Categorical(
        choices=(
            space.Resampled(_PARAMS),
            space.Resampled(_CONST),
            space.Resampled(_VAR),
        ),
    )

    _F_ARGS = space.Categorical(
        choices=(
            (
                space.Resampled(_UNARY_FUN),
                space.Resampled(_POINTER),
            ),
            (
                space.Resampled(_BINARY_FUN),
                space.Resampled(_POINTER),
                space.Resampled(_POINTER),
            ),
            (
                space.Resampled(_TERNARY_FUN),
                space.Resampled(_POINTER),
                space.Resampled(_POINTER),
                space.Resampled(_POINTER),
            ),
        ),
    )

    _F = space.Operation(
        operator=_function_writer,
        args=space.Resampled(_F_ARGS),
        kwargs={"store": space.Resampled(_VAR)}, # move to _F_ARGS as normal arg?
    )

    _P_ARGS = space.Categorical(
        choices=(
            (space.Resampled(_F),),
            (space.Resampled(_F), space.Resampled("program"),), # Can be multiple space.Resampled("P")
        ),
        prior_index=1, # Encourages longer programs
        prior_confidence=space.ConfidenceLevel.LOW
    )

    program = space.Operation(
        operator=program_compiler,
        args=space.Resampled(_P_ARGS),
    )

    epochs = space.Fidelity(space.Integer(1, MAX_EPOCHS_PER_CONFIG))


In [2]:
SGD = Program(
    [
        # momentum = 1 - 0.1 = 0.9
        Instruction(Function(torch.sub, 2), [Pointer(3), Pointer(5)], Pointer(9)),
        # m = m * momentum
        Instruction(Function(torch.mul, 2), [Pointer(10), Pointer(9)], Pointer(10)),
        # m = gradient + m
        Instruction(Function(torch.add, 2), [Pointer(1), Pointer(10)], Pointer(10)),

        Instruction(Function(torch.div, 2), [Pointer(10), Pointer(4)], Pointer(16)), # Unnecessary line, should be removed by pruning
        # update = m * 0.001
        Instruction(Function(torch.mul, 2), [Pointer(10), Pointer(6)], Pointer(11)),
    ]
)
pprint.pprint(SGD)
prune_program(SGD)
pprint.pprint(SGD)

# from nosbench.optimizers import AdamW
# for e in range(20):
#     benchmark = nosbench.NOSBench()
#     loss = benchmark.query(AdamW, epoch=e)
#     print(loss)
    

[Instruction(Function(sub, 2), inputs=[3, 5], output=9),
 Instruction(Function(mul, 2), inputs=[10, 9], output=10),
 Instruction(Function(add, 2), inputs=[1, 10], output=10),
 Instruction(Function(div, 2), inputs=[10, 4], output=16),
 Instruction(Function(mul, 2), inputs=[10, 6], output=11)]
[Instruction(Function(sub, 2), inputs=[3, 5], output=9),
 Instruction(Function(mul, 2), inputs=[10, 9], output=10),
 Instruction(Function(add, 2), inputs=[1, 10], output=10),
 Instruction(Function(mul, 2), inputs=[10, 6], output=11)]


In [ ]:
def evaluate_pipeline(program: Program, epochs:int=1) -> float:
    # benchmark = nosbench.NOSBench()
    benchmark = nosbench.ToyBenchmark()
    prune_program(program)
    objective_to_minimize = benchmark.query(program, epochs)
    assert isinstance(objective_to_minimize, float)
    objective_to_minimize = torch.inf if math.isnan(objective_to_minimize) else objective_to_minimize
    return objective_to_minimize


optimizers=[(space.RandomSearch,
            "new__RandomSearch",),
            (space.ComplexRandomSearch,
            "new__ComplexRandomSearch",),
            (partial(new_bracket_optimizer.priorband, base="successive_halving"),
                "new__priorband+successive_halving",),
            (partial(new_bracket_optimizer.priorband, base="asha"),
                "new__priorband+asha",),
            (partial(new_bracket_optimizer.priorband, base="async_hb"),
                "new__priorband+async_hb",),
            (new_bracket_optimizer.priorband,
                "new__priorband+hyperband",),
            ]


def nosbench_neps_demo(optimizer, optimizer_name):
    optimizer.__name__ = optimizer_name  # Needed by NEPS later.
    pipeline_space = nosBench()
    root_directory = f"results/nosbench_{optimizer.__name__}"

    print(f"Running for root directory: {root_directory}")

    neps.run(
        evaluate_pipeline=space.adjust_evaluation_pipeline_for_new_space(
            evaluate_pipeline,
            pipeline_space,
        ),
        pipeline_space=pipeline_space,
        optimizer=optimizer,
        root_directory=root_directory,
        post_run_summary=True,
        max_evaluations_total=200,
        overwrite_working_directory=True,
    )
    neps.status(root_directory, print_summary=True)

for optimizer in optimizers:
    print(f"\nRunning optimizer: {optimizer[1]}")
    nosbench_neps_demo(*optimizer)


Running optimizer: new__RandomSearch
Running for root directory: results/nosbench_new__RandomSearch
# Configs: 200

    success: 200

# Best Found (config 35):

    objective_to_minimize: 0.18430356979370116
    config: {'SAMPLING__Resolvable.program.args.resampled_categorical::categorical__2': 1, 'SAMPLING__Resolvable.program.args[0].resampled_operation.args.resampled_categorical::categorical__3': 2, 'SAMPLING__Resolvable.program.args[0].resampled_operation.kwargs{store}.resampled_integer::integer__9_20_False': 20, 'SAMPLING__Resolvable.program.args[0].resampled_operation.args[0].resampled_categorical::categorical__2': 1, 'SAMPLING__Resolvable.program.args[0].resampled_operation.args[1].resampled_categorical::categorical__3': 2, 'SAMPLING__Resolvable.program.args[0].resampled_operation.args[1].resampled_categorical.sampled_value.resampled_integer::integer__9_20_False': 20, 'SAMPLING__Resolvable.program.args[0].resampled_operation.args[2].resampled_categorical::categorical__3': 2, 'SA

In [4]:
# Load in config from yaml file in results folder
import yaml

with open("./results/nosbench_new__priorband+hyperband/configs/config_59_2/config.yaml") as stream:
    config=yaml.safe_load(stream)
    # pprint.pprint(config)

# For each key, remove the 'SAMPLING__' prefix, except for 'epochs'
for key in list(config.keys()):
    if key.startswith('SAMPLING__'):
        new_key = key[len('SAMPLING__'):]
        config[new_key] = config.pop(key)

# Collect the ENVIRONMENT__ values in a separate dictionary without the prefix
environment_values = {}
for key in list(config.keys()):
    if key.startswith('ENVIRONMENT__'):
        new_key = key[len('ENVIRONMENT__'):]
        environment_values[new_key] = config.pop(key)

resolved_pipeline, resolution_context = space.resolve(nosBench(),space.OnlyPredefinedValuesSampler(config),environment_values=environment_values)
program=space.convert_operation_to_callable(resolved_pipeline.program)
pprint.pprint(program)
prune_program(program)
print()
pprint.pprint(program)

[Instruction(Function(bias_correct, 3), inputs=[1, 13, 0], output=15),
 Instruction(Function(interpolate, 3), inputs=[0, 15, 17], output=17),
 Instruction(Function(minimum, 2), inputs=[9, 14], output=12),
 Instruction(Function(minimum, 2), inputs=[19, 16], output=10),
 Instruction(Function(interpolate, 3), inputs=[20, 1, 17], output=16)]

[Instruction(Function(bias_correct, 3), inputs=[1, 13, 0], output=15),
 Instruction(Function(interpolate, 3), inputs=[0, 15, 17], output=17),
 Instruction(Function(interpolate, 3), inputs=[20, 1, 17], output=16)]
